# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397834415952                   -0.90    5.0   26.7ms
  2   -8.400245915271       -2.62       -1.73    1.0   18.8ms
  3   -8.400402020826       -3.81       -2.93    1.5   19.5ms
  4   -8.400427767898       -4.59       -2.91    3.2   25.0ms
  5   -8.400427954841       -6.73       -3.04    1.0   53.6ms
  6   -8.400428149837       -6.71       -4.88    1.0   18.8ms
  7   -8.400428155853       -8.22       -4.43    3.2   25.6ms
  8   -8.400428156260       -9.39       -5.77    1.0   19.2ms
  9   -8.400428156276      -10.79       -6.21    2.0   22.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397728855503                   -0.90           5.0    558ms
  2   -8.400384528665       -2.58       -1.79   0.80    2.2    519ms
  3   -8.400423208492       -4.41       -3.02   0.80    1.0    191ms
  4   -8.400428116584       -5.31       -3.47   0.80    2.8   21.4ms
  5   -8.400428152971       -7.44       -4.67   0.80    1.0   16.4ms
  6   -8.400428156261       -8.48       -5.96   0.80    2.8   21.8ms
  7   -8.400428156277      -10.81       -6.41   0.80    3.2   22.8ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +0.846180307029                   -1.08    2.87s
  2   -1.784578630584        0.42       -0.62    108ms
  3   -3.597345251466        0.26       -0.39   39.3ms
  4   -4.629740099205        0.01       -0.45   87.7ms
  5   -6.320352035011        0.23       -0.50   39.5ms
  6   -7.125459740811       -0.09       -0.68   39.5ms
  7   -7.885911603677       -0.12       -0.90   39.2ms
  8   -8.056020732470       -0.77       -1.51   50.3ms
  9   -8.163188357514       -0.97       -1.81   29.1ms
 10   -8.207796960236       -1.35       -1.91   29.1ms
 11   -8.257183587233       -1.31       -2.09   29.1ms
 12   -8.298237362961       -1.39       -1.74   29.0ms
 13   -8.334447083358       -1.44       -1.91   28.9ms
 14   -8.369960309394       -1.45       -2.05   33.4ms
 15   -8.389194688309       -1.72       -2.39   29.1ms
 16   -8.396577635833       -2.13       -2.47   29.3ms
 17   -8.39

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397868493542                   -0.90    5.2   27.1ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427988174                   -1.79    24.2s
  2   -8.400428156277       -6.77       -4.03    2.19s
  3   -8.400428156277      -14.45       -7.84   97.9ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 8.188620488572257e-7
|ρ_newton - ρ_scfv| = 8.629177351446814e-8
|ρ_newton - ρ_dm|   = 9.043735642903614e-7
